In [1]:
import pandas as pd
import pickle
from helper_functions import Helper
from helper_functions import Coinmetrics

In [ ]:
if __name__ == "__main__":
    # set args
    CW_IN_FP = '../data/derived/cm_to_coinapi_cw.pkl'
    ASSET_IN_FP = '../data/clean/asset_universe_dict.pickle'
    CM_API_FP = '../../admin/coinmetrics.txt'
    BASE_URL = 'https://api.coinmetrics.io/v4/'
    PANEL_OUT_FP = '../data/raw/coinmetrics_panel_hourly.pkl'
    MACRO_OUT_FP = '../data/raw/coinmetrics_macro_hourly.pkl'
    TARGET_US_EXCHANGES = ['binance.us',  'bitstamp', 'coinbase', 'crypto.com', 'ftx.us', 
        'gemini', 'kraken', 'kucoin']
    TARGET_EXCHANGE_METRICS = ['open_interest_reported_future_usd', 'volume_reported_future_usd_1h', 'volume_reported_spot_usd_1h']
    TARGET_FREQ = '1h'
    STUDY_START = '2016-07-01'
    STUDY_END = '2023-01-03'
    TARGET_METRICS = ['AdrActCnt', 'AdrActContCnt', 'AdrActRecCnt', 'AdrActSentCnt', 'AdrBalCnt',
        'AdrBalUSD100Cnt', 'AdrBalUSD100KCnt', 'AdrBalUSD10Cnt', 'AdrBalUSD10KCnt', 'AdrBalUSD10MCnt',
        'AdrBalUSD1Cnt', 'AdrBalUSD1KCnt', 'AdrBalUSD1MCnt', 'CapAct1yrUSD',
        'CapFutExp10yrUSD', 'CapMVRVCur', 'CapMVRVFF', 'CapMrktCurUSD', 'CapMrktEstUSD', 'CapMrktFFUSD',
        'CapRealUSD', 'ContERC1155Cnt', 'ContERC20Cnt', 'ContERC721Cnt',
        'DiffMean', 'FeeMeanUSD', 'FeeMedUSD', 'FeeRevPct', 'FeeTotUSD', 'FlowInExInclUSD', 'FlowInExUSD',
        'FlowMinerIn0HopAllUSD', 'FlowMinerIn1HopAllUSD', 'FlowMinerNet0HopAllUSD', 'FlowMinerNet1HopAllUSD',
        'FlowMinerOut0HopAllUSD', 'FlowMinerOut1HopAllUSD', 'FlowOutExInclUSD', 'FlowOutExUSD', 'FlowTfrFromExCnt',
        'FlowTfrFromExInclCnt', 'FlowTfrToExCnt', 'FlowTfrToExInclCnt', 'FlowToCLCont', 'GasUsedTx', 'GasUsedTxMean',
        'HashRate', 'IssContNtv', 'IssContPctAnn', 'IssContPctDay', 'IssContUSD', 'IssTotUSD', 'MCRC', 'MCTC',
        'MOMR', 'MRI0HopAll30d', 'MRI1HopAll30d', 'NDF', 'NVTAdj', 'NVTAdjFF', 'PriceUSD', 'PuellMulCont', 'PuellMulRev',
        'PuellMulTot', 'RCTC', 'RVTAdj', 'ReferenceRateUSD', 'RevAllTimeUSD', 'RevHash1yAvgNtv', 'RevHash1yAvgUSD',
        'RevHashUSD', 'RevUSD', 'SER', 'SOPR', 'SOPROut', 'SenderCntCLCont', 'SenderTotCLCont', 'SplyAct10yr',
        'SplyAct180d', 'SplyAct1d', 'SplyAct1yr', 'SplyAct2yr', 'SplyAct30d', 'SplyAct3yr', 'SplyAct4yr', 'SplyAct5yr',
        'SplyAct7d', 'SplyAct90d', 'SplyActEver', 'SplyActPct1yr', 'SplyAdrBalUSD1', 'SplyAdrBalUSD10',
        'SplyAdrBalUSD100', 'SplyAdrBalUSD100K', 'SplyAdrBalUSD10K', 'SplyAdrBalUSD10M', 'SplyAdrBalUSD1K',
        'SplyAdrBalUSD1M', 'SplyAdrTop100', 'SplyAdrTop10Pct', 'SplyAdrTop1Pct', 'SplyBurntUSD', 'SplyContUSD',
        'SplyCur', 'SplyExUSD', 'SplyExpFut10yr', 'SplyFF', 'SplyMiner0HopAllUSD', 'SplyMiner1HopAllUSD',
        'SplyRvv180d', 'SplyRvv1yr', 'SplyRvv2yr', 'SplyRvv30d', 'SplyRvv3yr', 'SplyRvv4yr', 'SplyRvv5yr', 
        'SplyRvv7d', 'SplyRvv90d', 'SplyUTXOLoss', 'SplyUTXOProf', 'TxCnt', 'TxTfrCnt', 'TxTfrValAdjUSD',    
        'TxTfrValContCallUSD', 'TxTfrValDayDst', 'TxTfrValDayDstMean', 'TxTfrValMeanUSD', 'TxTfrValMedUSD',
        'TxTfrValNtv', 'TxTfrValRecNtv', 'TxTfrValSentNtv', 'TxTfrValUSD', 'UTXOAgeMean', 'UTXOAgeMed', 
        'UTXOAgeValMean', 'UTXOCnt', 'UTXODay', 'UTXOLossCnt', 'UTXOLossUnrealUSD', 'UTXOProfCnt',
        'UTXOProfUnrealUSD', 'VelAct1yr', 'VelCur1yr', 'WalActCnt', 'WalActRecCnt', 'WalActSentCnt', 'WalBalCnt',
        'mempool_count', 'mempool_fee', 'mempool_fee_mean', 'mempool_fee_median', 'mempool_feerate_mean',  
        'mempool_feerate_median', 'mempool_output_value', 'mempool_size', 'mempool_vsize', 
        'mining_reward_mean', 'mining_reward_spread']
    
    # import api key
    with open(CM_API_FP) as f:
        API_KEY = f.readlines()
        API_KEY = API_KEY[0].strip()
    BASE_PARAMS = {'api_key': API_KEY}

    # Import asset universe and cw
    cw_df = pd.read_pickle(CW_IN_FP)
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    asset_universe = Helper.findUniqueAssets(asset_universe_dict)

    # Pull info on assets
    assets_df = Coinmetrics.pullAssetInfo(BASE_URL, BASE_PARAMS, asset_universe+['usdt', 'usdc'])

    # pull meta data on target exchanges
    exchanges_df = Coinmetrics.pullExchangeInfo(BASE_URL, BASE_PARAMS, TARGET_US_EXCHANGES)

    # pull meta data markets and subset down to target markets
    markets_df = Coinmetrics.pullAndFormRelevantMarkets(exchanges_df, assets_df, BASE_URL, BASE_PARAMS, asset_universe)

    # pull reference rates for USDC and USDT to use for exchange rate calculations
    usd_df = Coinmetrics.pullAssetMetrics(BASE_URL, BASE_PARAMS, assets_df, STUDY_START, STUDY_END, 
        TARGET_FREQ, ['usdc', 'usdt'], ['ReferenceRateUSD'])
    usd_df = Coinmetrics.cleanUSDPrices(usd_df, TARGET_FREQ)

    # pull prices and volumes for all markets of interest
    ohlcv_df = Coinmetrics.pullOHLCV(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, TARGET_FREQ, markets_df)

    # pull asset metrics
    panel_df = Coinmetrics.pullAssetMetrics(BASE_URL, BASE_PARAMS, assets_df, STUDY_START, STUDY_END,
                                            TARGET_FREQ, asset_universe, TARGET_METRICS)
    panel_df = Coinmetrics.cleanPanel(panel_df)

    # pull exchange metrics
    ex_metrics_df = Coinmetrics.pullExchangeMetrics(BASE_URL, BASE_PARAMS, TARGET_EXCHANGE_METRICS, 
        STUDY_START, STUDY_END, TARGET_FREQ)
    ex_metrics_df = Coinmetrics.cleanExchangeMetrics(ex_metrics_df, TARGET_EXCHANGE_METRICS, TARGET_US_EXCHANGES)

    # form macro dataframe
    macro_df = usd_df.merge(ex_metrics_df, on=['date'], how='outer', validate='one_to_one')
    macro_df.to_pickle(MACRO_OUT_FP)

    # form the panel
    # prices_df = Coinmetrics.cleanMarketPanels(ohlcv_df, TARGET_FREQ, markets_df, usd_df, ba_df)
    # panel_df = panel_df.merge(prices_df, on=['date', 'asset'], how='outer', validate='one_to_one')
    # panel_df = panel_df.sort_values(by=['date', 'asset'], ignore_index=True)
    # panel_df.to_pickle(PANEL_OUT_FP)
    # TODO UNCOMMENT 4 LINES ABOVE BEFORE FINAL SAVE

In [28]:
# TODO pull bid ask prices


In [30]:
# TODO 1
markets_df = master_markets_df.copy()
end   = int(1*markets_df.shape[0]/10)
markets_df = markets_df[:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_1.pkl')
del ba_df
import gc
gc.collect()

The API call failed with error: 429 Client Error: Too Many Requests for url: https://api.coinmetrics.io/v4/timeseries/market-quotes?api_key=hMkOUrQIV5EvIGiRyxZ9&page_size=10000&limit_per_market=10000&end_inclusive=False&markets=binance.us-ach-usdt-spot&start_time=2022-10-24&end_time=2022-10-25
Retrying after 1.11 seconds.


In [ ]:
import time
time.sleep(60*5)

In [ ]:
# TODO 2
markets_df = master_markets_df.copy()
start = int(1*markets_df.shape[0]/10)
end   = int(2*markets_df.shape[0]/10)
markets_df = markets_df[start:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_2.pkl')
del ba_df
gc.collect()

In [ ]:

time.sleep(60*5)

In [ ]:
# TODO 3
markets_df = master_markets_df.copy()
start = int(2*markets_df.shape[0]/10)
end   = int(3*markets_df.shape[0]/10)
markets_df = markets_df[start:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_3.pkl')
del ba_df
gc.collect()

In [ ]:

time.sleep(60*5)

In [ ]:
# TODO 4
markets_df = master_markets_df.copy()
start = int(3*markets_df.shape[0]/10)
end   = int(4*markets_df.shape[0]/10)
markets_df = markets_df[start:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_4.pkl')
del ba_df
gc.collect()

In [ ]:

time.sleep(60*5)

In [ ]:
# TODO 5
markets_df = master_markets_df.copy()
start = int(4*markets_df.shape[0]/10)
end   = int(5*markets_df.shape[0]/10)
markets_df = markets_df[start:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_5.pkl')
del ba_df
gc.collect()

In [ ]:

time.sleep(60*5)

In [ ]:
# TODO 6
markets_df = master_markets_df.copy()
start = int(5*markets_df.shape[0]/10)
end   = int(6*markets_df.shape[0]/10)
markets_df = markets_df[start:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_6.pkl')
del ba_df
gc.collect()

In [ ]:

time.sleep(60*5)

In [ ]:
# TODO 7
markets_df = master_markets_df.copy()
start = int(6*markets_df.shape[0]/10)
end   = int(7*markets_df.shape[0]/10)
markets_df = markets_df[start:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_7.pkl')
del ba_df
gc.collect()

In [ ]:

time.sleep(60*5)

In [ ]:
# TODO 8
markets_df = master_markets_df.copy()
start = int(7*markets_df.shape[0]/10)
end   = int(8*markets_df.shape[0]/10)
markets_df = markets_df[start:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_8.pkl')
del ba_df
gc.collect()

In [ ]:

time.sleep(60*5)

In [ ]:
# TODO 9
markets_df = master_markets_df.copy()
start = int(8*markets_df.shape[0]/10)
end   = int(9*markets_df.shape[0]/10)
markets_df = markets_df[start:end]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_9.pkl')
del ba_df
gc.collect()

In [ ]:

time.sleep(60*5)

In [ ]:
# TODO 10
markets_df = master_markets_df.copy()
start = int(9*markets_df.shape[0]/10)
markets_df = markets_df[start:]
ba_df = Coinmetrics.pullBidAsk(BASE_URL, BASE_PARAMS, STUDY_START, STUDY_END, markets_df)
ba_df.to_pickle('temp_ba_10.pkl')
del ba_df
gc.collect()

In [ ]:
# TODO CHECK THAT I GOT ALL MARKETS?

In [47]:
# TODO IMPORT ALL BELOW DF'S FROM TEMP IN CODE FOLDER
ohlcv_df = pd.read_pickle('temp_ohlcv.pkl')
markets_df = pd.read_pickle('temp_markets.pkl')
usd_df =  pd.read_pickle('temp_usd.pkl')
panel_df = pd.read_pickle('temp_panel.pkl')
ba_df1 = pd.read_pickle('temp_ba_1.pkl')
ba_df2 = pd.read_pickle('temp_ba_2.pkl')
ba_df3 = pd.read_pickle('temp_ba_3.pkl')
ba_df4 = pd.read_pickle('temp_ba_4.pkl')
ba_df5 = pd.read_pickle('temp_ba_5.pkl')
ba_df6 = pd.read_pickle('temp_ba_6.pkl')
ba_df7 = pd.read_pickle('temp_ba_7.pkl')
ba_df8 = pd.read_pickle('temp_ba_8.pkl')
ba_df9 = pd.read_pickle('temp_ba_9.pkl')
ba_df10 = pd.read_pickle('temp_ba_10.pkl')
ba_df = pd.concat([ba_df1, ba_df2, ba_df3, ba_df4,
                ba_df5, ba_df6, ba_df7, ba_df8, ba_df9, ba_df10])

del ba_df1
del ba_df2
del ba_df3
del ba_df4
del ba_df5
del ba_df6
del ba_df7
del ba_df8
del ba_df9
del ba_df10
import gc
gc.collect()

478

In [57]:
# form the panel
prices_df = Coinmetrics.cleanMarketPanels(ohlcv_df, TARGET_FREQ, markets_df, usd_df, ba_df)
master_panel_df = panel_df.copy()
panel_df = panel_df.merge(prices_df, on=['date', 'asset'], how='outer', validate='one_to_one')
panel_df = panel_df.sort_values(by=['date', 'asset'], ignore_index=True)
panel_df.to_pickle(PANEL_OUT_FP)